In [1]:
import numpy as np
import pandas as pd

###  Fonction de prédiction KNN :

In [2]:
pd.options.mode.chained_assignment = None
np.random.seed(1)

df_airbnb_paris = pd.read_csv('paris_airbnb.csv')      ### lire le fichier .csv  ###
#x = (len(df_airbnb_paris * 75/100))
#print(x)
sup_virgule = df_airbnb_paris['price'].str.replace(',','')    ###  replace la virgule par un vide  ###
sup_dollars = sup_virgule.str.replace('$','')         ###  replace le signe $ par un vide  ###
df_airbnb_paris['price'] = pd.to_numeric(sup_dollars, downcast = 'float')  ###  
df_airbnb_paris = df_airbnb_paris.loc[np.random.permutation(len(df_airbnb_paris))]
train_df = df_airbnb_paris.iloc[0:6000]
test_df = df_airbnb_paris.iloc[6000:]
def predict_prix (k):
    tmp = train_df.copy()
    tmp['distance'] = tmp['accommodates'].apply(lambda x: np.abs(x - k))
    tmp = tmp.sort_values('distance')
    nearest_neighbors = tmp.iloc[0 : 300]['price']
    prix_predit = nearest_neighbors.mean()
    
    return prix_predit

p1 = predict_prix(1)
p2 = predict_prix(2)
p3 = predict_prix(3)
p4 = predict_prix(4)

print(p1);print(p2);print(p3);print(p4)

test_df['prix_predit'] = test_df['accommodates'].apply(lambda x: predict_prix(x))
#test_df.head()
test_df['erreur'] = np.absolute(test_df['prix_predit'] - test_df['price']) ## pour calculer erreur absolue moyenne ##
MAE = test_df['erreur'].mean()
print(MAE)

test_df['erreur_quadratique'] = (test_df['prix_predit']- test_df ['price']) ** 2
MSE = test_df['erreur_quadratique'].mean()
RMSE = MSE **(1/2)
print(MSE)
print(RMSE)

64.88999938964844
82.61000061035156
101.3499984741211
126.586669921875
43.83414703369141
5913.683659442962
76.9004789285669


##  Modèle multivariée des K plus proche voisin
### 1° : Suprimer des caractéristiques

In [3]:
df_airbnb_paris.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 4740 to 5157
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   host_response_rate    5000 non-null   object 
 1   host_acceptance_rate  0 non-null      float64
 2   host_listings_count   7999 non-null   float64
 3   latitude              8000 non-null   float64
 4   longitude             8000 non-null   float64
 5   city                  7997 non-null   object 
 6   zipcode               7930 non-null   object 
 7   state                 7977 non-null   object 
 8   accommodates          8000 non-null   int64  
 9   room_type             8000 non-null   object 
 10  bedrooms              7976 non-null   float64
 11  bathrooms             7942 non-null   float64
 12  beds                  7986 non-null   float64
 13  price                 8000 non-null   float32
 14  cleaning_fee          6250 non-null   object 
 15  security_deposit  

In [4]:
drop_colonne = ['host_response_rate','host_acceptance_rate','host_listings_count','latitude','longitude','city','zipcode','state','room_type']
df_airbnb_paris = df_airbnb_paris.drop(drop_colonne,axis=1) # supprimer les 9 colonnes de dataframes 
print(df_airbnb_paris.isnull().sum()) ## isnull()il determine si y a la valeur null; sum()il va sommer les valeur manquante

accommodates            0
bedrooms               24
bathrooms              58
beds                   14
price                   0
cleaning_fee         1750
security_deposit     1680
minimum_nights          0
maximum_nights          0
number_of_reviews       0
dtype: int64


### 2°: Gérer les valeurs manquantes:

In [5]:
supp_collone = ['cleaning_fee','security_deposit'] # suppression des colonnes qui ont bcp de valeur manquante
df_airbnb_paris = df_airbnb_paris.drop(supp_collone,axis=1)
df_airbnb_paris = df_airbnb_paris.dropna(axis=0) # suppression des lignes qui contient les valeurs manquante
print(df_airbnb_paris.isnull().sum())
print()
df_airbnb_paris.head(6)

accommodates         0
bedrooms             0
bathrooms            0
beds                 0
price                0
minimum_nights       0
maximum_nights       0
number_of_reviews    0
dtype: int64



,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
4740,4,1.0,1.0,2.0,65.0,2,1125,6
5606,3,2.0,1.5,2.0,98.0,5,1124,1
4824,2,0.0,1.0,1.0,65.0,10,1125,0
4205,2,1.0,1.0,1.0,45.0,5,1125,83
3228,3,1.0,1.0,2.0,65.0,5,365,5
2745,4,1.0,1.0,2.0,88.0,3,30,120


### 3°: Normaliser les colonnes:

In [6]:
normalisation_airbnb_paris = (df_airbnb_paris - df_airbnb_paris.mean())/df_airbnb_paris.std()
normalisation_airbnb_paris['price'] = df_airbnb_paris['price']
normalisation_airbnb_paris.head(6)

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
4740,0.503868,-0.296884,-0.293537,0.20531,65.0,-0.184601,1.062858,-0.564545
5606,-0.131849,0.892605,0.843973,0.20531,98.0,-0.101183,1.061018,-0.636924
4824,-0.767566,-1.486372,-0.293537,-0.64526,65.0,0.037847,1.062858,-0.651400
4205,-0.767566,-0.296884,-0.293537,-0.64526,45.0,-0.101183,1.062858,0.550105
3228,-0.131849,-0.296884,-0.293537,0.20531,65.0,-0.101183,-0.335685,-0.579021
2745,0.503868,-0.296884,-0.293537,0.20531,88.0,-0.156795,-0.952148,1.085716


### 4°: distance euclidienne pour le cas multivarié

In [7]:
from scipy.spatial import distance
l1_normalise = normalisation_airbnb_paris.iloc[0][['accommodates','bedrooms']]
l2_normalise = normalisation_airbnb_paris.iloc[4][['accommodates','bedrooms']]
dist = distance.euclidean(l1_normalise,l2_normalise)
print(dist)

0.6357172321498359


# Scikit_learn

## classe KneighborsRegresors
###  1°: Adapter le modèle et faire des prédiction

In [12]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=5,algorithm='brute') ## instancier le modèle



train_df = normalisation_airbnb_paris.iloc[0:6000]
test_df = normalisation_airbnb_paris.iloc[6000:]

train_colonne =train_df[['accommodates','bedrooms','bathrooms','number_of_reviews','beds']]

colonne_cible = train_df['price']
knn.fit(train_colonne,colonne_cible)

test_colonne = test_df[['accommodates','bedrooms','bathrooms','number_of_reviews','beds']]
predict = knn.predict(test_colonne) 
predict

array([ 90.2,  90.2,  90.2, ...,  90.2, 141.8,  90.2], dtype=float32)

### 2°: calcule de l'erreur quadratique moyenne

In [13]:
from sklearn.metrics import mean_squared_error
a = test_df['price'].to_numpy()
sk_mse = mean_squared_error(a,predict)
sk_rmse = sk_mse**(1/2)
print(sk_mse)
print(sk_rmse)

7744.82
88.00465796887543
